# ATUS Hierarchical Baseline Experiments - HPC Version

## Overview

This notebook runs the ATUS (American Time Use Survey) hierarchical baseline experiments safely on HPC systems. It includes 7 individual experiment rungs (R1-R7) that can be run independently.

## Experiment Structure

- **R1**: Region only
- **R2**: Region + Sex
- **R3**: Region + Employment
- **R4**: Region + Day Type
- **R5**: Region + Household Size Band
- **R6**: Full routing model (Employment + Day Type + HH Size + Sex + Region + Quarter)
- **R7**: Full model with hazard (same grouping as R6 but includes hazard modeling)

## How to Use This Notebook

1. **Run Setup Cells**: Execute cells 1-3 to import libraries and set up the environment
2. **Check System Resources**: Run cell 4 to verify your HPC node has sufficient resources
3. **Run Individual Experiments**: Execute cells 5-11 one at a time for each rung (R1-R7)
4. **Monitor Progress**: Each cell will show detailed progress and can be interrupted safely
5. **Resume if Needed**: If interrupted, you can restart from any cell - completed experiments won't be re-run

## Expected Runtime

- **R1-R4**: 30-60 minutes each
- **R5-R6**: 60-120 minutes each  
- **R7**: 120-180 minutes (includes hazard model)
- **Total**: 6-12 hours for all experiments

## Resource Requirements

- **Memory**: At least 16GB RAM recommended
- **Storage**: At least 20GB free disk space
- **CPU**: Multi-core recommended for faster processing

## Import Required Libraries

In [ ]:
import sys
import os
import pandas as pd
from pathlib import Path
import subprocess
import time
from datetime import datetime
import logging
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Setup paths - this notebook should be in scripts/ folder  
scripts_dir = Path.cwd()
project_root = scripts_dir.parent  
sys.path.insert(0, str(project_root))

from scripts.run_single_experiments import run_single_experiment
from scripts.baseline1_hier import run_baseline1_hier 
from scripts.baseline2_hier import run_baseline2_hier

print("Helper functions defined with improved direct execution support")

✓ Libraries imported successfully
Python version: 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]
Working directory: /ztank/scratch/user/u.rd143338/atus_analysis-main


## Define Experiment Configuration

In [ ]:
def run_experiment_safely(name, func):
    """
    Safely run an experiment with error handling and memory management.
    """
    try:
        print(f"\n--- Running {name} ---")
        start_time = time.time()
        
        # Run the experiment
        func()
        
        elapsed = time.time() - start_time
        print(f"{name} completed in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
        return True
        
    except Exception as e:
        print(f"ERROR in {name}: {str(e)}")
        print("Details:")
        import traceback
        traceback.print_exc()
        
        # Check for common error patterns
        error_str = str(e).lower()
        if 'memory' in error_str or 'out of memory' in error_str:
            print("WARNINGS:")
            print("- Memory error detected. Try reducing batch size or closing other applications.")
        elif 'cuda' in error_str:
            print("WARNINGS:")
            print("- CUDA error detected. GPU memory might be full.")
        elif 'file not found' in error_str or 'no such file' in error_str:
            print("WARNINGS:")  
            print("- File not found. Check that all prerequisite steps completed successfully.")
        
        return False

print("Helper functions defined with improved direct execution support")

✓ Configuration set
Output directory: atus_analysis/data/models
Number of rungs: 7


## Define Helper Functions

In [ ]:
def run_single_experiment_wrapper(rung, dataset_path, args, compute_expected=True):
    """
    Enhanced wrapper that handles direct execution of run_single_experiment with better error handling.
    """
    try:
        # Convert rung to string for run_single_experiment
        rung_str = f"R{rung}"
        
        print(f"Loading data for {rung_str}...")
        
        # Load data  
        df = pd.read_parquet(dataset_path)
        
        # Check if 'quarter' exists, if not use 'month' and derive quarter
        if 'quarter' not in df.columns:
            if 'month' in df.columns:
                df['quarter'] = df['month'].apply(lambda x: (x-1)//3 + 1)
                print(f"Derived quarter from month column")
            else:
                raise ValueError("Neither quarter nor month column found!")
        
        # Generate or load splits
        from pathlib import Path
        models_dir = Path(args.output_dir) / "models" / rung_str
        split_path = models_dir / "fixed_split.parquet"
        
        if split_path.exists():
            print(f"Loading existing split from {split_path}")
            split_df = pd.read_parquet(split_path)
            # Extract train/test splits  
            train_ids = split_df[split_df['split'] == 'train']['caseid'].tolist()
            test_ids = split_df[split_df['split'] == 'test']['caseid'].tolist()
        else:
            print(f"Generating new train/test split for {rung_str}")
            # Generate splits using original logic
            unique_cases = df['caseid'].unique()
            n_train = int(0.8 * len(unique_cases))
            np.random.seed(42)  # For reproducibility
            train_ids = np.random.choice(unique_cases, size=n_train, replace=False).tolist()
            test_ids = [cid for cid in unique_cases if cid not in train_ids]
            
            # Save split
            split_data = []
            for cid in train_ids:
                split_data.append({'caseid': cid, 'split': 'train'})
            for cid in test_ids:
                split_data.append({'caseid': cid, 'split': 'test'})
            
            split_df = pd.DataFrame(split_data)
            models_dir.mkdir(parents=True, exist_ok=True)
            split_df.to_parquet(split_path)
            print(f"Saved split to {split_path}")
        
        # Now run the experiment with direct execution
        print(f"--- Step 1: B1-H Model for {rung_str} ---")
        
        try:
            # Call B1-H directly
            run_baseline1_hier(
                data=df,
                train_ids=train_ids,
                test_ids=test_ids,
                rung=rung_str,
                output_dir=args.output_dir,
                compute_expected=compute_expected,
                verbose=True
            )
            print(f"B1-H model completed successfully for {rung_str}")
            
        except Exception as e:
            print(f"Direct execution failed: {e}")
            
            # Fallback to subprocess if direct execution fails
            print(f"Falling back to subprocess execution for {rung_str}")
            cmd = [
                sys.executable, "-m", "scripts.baseline1_hier",
                "--rung", rung_str,
                "--output_dir", args.output_dir,
                "--dataset_path", dataset_path
            ]
            if compute_expected:
                cmd.append("--compute_expected")
                
            process = subprocess.run(cmd, cwd=project_root, capture_output=True, text=True)
            if process.returncode == 0:
                print(f"B1-H model completed successfully for {rung_str}")
            else:
                print(f"B1-H model failed for {rung_str} (return code: {process.returncode})")
                print("STDOUT:", process.stdout[-1000:])  # Last 1000 chars
                print("STDERR:", process.stderr[-1000:])
                raise Exception(f"B1-H subprocess failed with return code {process.returncode}")
                
    except Exception as e:
        print(f"Subprocess execution failed: {e}")
        raise

def run_baseline2_hier_wrapper(rung, dataset_path, args, compute_expected=True):
    """
    Wrapper for B2-H baseline (subprocess only - no direct execution due to complexity)
    """
    try:
        rung_str = f"R{rung}"
        print(f"--- Step 2: B2-H Model for {rung_str} ---")
        
        cmd = [
            sys.executable, "-m", "scripts.baseline2_hier", 
            "--rung", rung_str,
            "--output_dir", args.output_dir,
            "--dataset_path", dataset_path
        ]
        if compute_expected:
            cmd.append("--compute_expected")
            
        process = subprocess.run(cmd, cwd=project_root, capture_output=True, text=True)
        if process.returncode == 0:
            print(f"B2-H model completed successfully for {rung_str}")
        else:
            print(f"B2-H model failed for {rung_str} (return code: {process.returncode})")
            print("STDOUT:", process.stdout[-1000:])  # Last 1000 chars  
            print("STDERR:", process.stderr[-1000:])
            raise Exception(f"B2-H subprocess failed with return code {process.returncode}")
            
    except Exception as e:
        print(f"B2-H execution failed: {e}")
        raise

def run_full_experiment_wrapper(rung, dataset_path, args, compute_expected=True):
    """
    Run both B1-H and B2-H for a given rung with enhanced monitoring
    """
    rung_str = f"R{rung}"
    print(f"STARTING RUNG {rung_str}")
    start_time = time.time()
    
    # Check if already completed
    models_dir = Path(args.output_dir) / "models" / rung_str
    expected_files = ["b1h_results.csv", "b2h_results.csv"]
    
    if all((models_dir / f).exists() for f in expected_files):
        print(f"{rung_str} already completed - skipping")
        return
    
    try:
        # Check memory and dataset size
        import psutil
        memory = psutil.virtual_memory()
        print(f"Available memory: {memory.available / (1024**3):.1f} GB")
        
        df = pd.read_parquet(dataset_path)
        print(f"Dataset size: {len(df):,} rows, {df.memory_usage(deep=True).sum() / (1024**3):.2f} GB")
        
        if memory.available < 8 * (1024**3):  # Less than 8GB available
            print("WARNING: Low memory detected. Consider closing other applications.")
        
        # Step 1: B1-H  
        print(f"--- Step 1: B1-H Model for {rung_str} ---")
        run_single_experiment_wrapper(rung, dataset_path, args, compute_expected)
        
        # Clear some memory between steps
        import gc
        gc.collect()
        
        # Step 2: B2-H (optional - comment out if having memory issues)
        # run_baseline2_hier_wrapper(rung, dataset_path, args, compute_expected)
        
        elapsed = time.time() - start_time
        print(f"{rung_str} COMPLETED SUCCESSFULLY in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
        
    except Exception as e:
        elapsed = time.time() - start_time
        print(f"ERROR in {rung_str} after {elapsed:.1f} seconds: {str(e)}")
        raise

✅ Helper functions defined with improved direct execution support


In [ ]:
# Check completion status from previous runs
completed = progress.get('completed_rungs', [])
failed = progress.get('failed_rungs', [])

print(f"Completed rungs: {completed if completed else 'None'}")
print(f"Failed rungs: {failed if failed else 'None'}")
print(f"Remaining rungs: {[r for r in RUNG_SPECS.keys() if r not in completed]}")

# Overall status
print("\n=== Overall Status ===")
if resources_ok and files_ok:
    print("READY TO START EXPERIMENTS")
    print("\nYou can now run the individual experiment cells below.")
else:
    print("PREREQUISITES NOT MET")
    if not resources_ok:
        print("   - System resources may be insufficient")
    if not files_ok:
        print("   - Required files are missing")
    print("\nPlease resolve issues before continuing.")

In [30]:
def run_baseline1_hier_direct(rung, groupby, output_dir, split_path):
    """Run baseline1_hier directly in Jupyter (preferred method)."""
    try:
        import pandas as pd
        import numpy as np
        from atus_analysis.scripts.common_hier import (
            prepare_long_with_groups, pool_rare_quarter,
            save_json, nll_b1, fit_b1_hier, parse_time_blocks
        )
        
        print(f"📊 Loading data for {rung}...")
        
        # Load sequences and subgroups
        sequences = pd.read_parquet(SEQUENCES_FILE)
        subgroups = pd.read_parquet(SUBGROUPS_FILE)
        
        # Add quarter column if needed for R6/R7 experiments
        subgroups = add_quarter_column(subgroups)
        
        print(f"✓ Loaded {len(sequences)} sequences and {len(subgroups)} subgroups")
        
        # Parse time blocks
        time_blocks = parse_time_blocks(TIME_BLOCKS)
        print(f"✓ Parsed time blocks: {time_blocks}")
        
        # Create or load split
        if split_path.exists():
            print(f"📂 Loading existing split from {split_path}")
            split_df = pd.read_parquet(split_path)
        else:
            print(f"🎲 Creating new split with seed {SEED}")
            # Create split logic here (simplified)
            np.random.seed(SEED)
            unique_ids = subgroups['TUCASEID'].unique()
            test_size = int(len(unique_ids) * TEST_SIZE)
            test_ids = np.random.choice(unique_ids, test_size, replace=False)
            
            split_df = pd.DataFrame({
                'TUCASEID': subgroups['TUCASEID'].unique(),
                'set': ['test' if id in test_ids else 'train' for id in subgroups['TUCASEID'].unique()]
            })
            split_df.to_parquet(split_path, index=False)
            print(f"✓ Split saved to {split_path}")
        
        # Prepare data with groups - fix the function call
        print(f"🔄 Preparing data with groupby: {groupby}")
        groupby_cols = groupby.split(',')
        
        # Call with correct signature including blocks parameter
        long_df = prepare_long_with_groups(sequences, subgroups, groupby_cols, time_blocks)
        
        # Pool rare quarters
        print(f"🔄 Pooling rare quarter groups...")
        long_df = pool_rare_quarter(long_df)
        
        # Merge with split
        print(f"🔄 Merging with train/test split...")
        long_df = long_df.merge(split_df, on='TUCASEID', how='left')
        
        print(f"📈 Fitting B1-H model...")
        # Fit the model
        result = fit_b1_hier(long_df)
        
        # Save results
        output_file = output_dir / "b1h_model.json"
        save_json(result, output_file)
        
        # Save evaluation
        eval_file = output_dir / "eval_b1h.json"
        test_data = long_df[long_df['set'] == 'test']
        eval_result = {
            'test_nll': nll_b1(result['params'], test_data),
            'n_test_sequences': len(test_data['TUCASEID'].unique()),
            'n_train_sequences': len(long_df[long_df['set'] == 'train']['TUCASEID'].unique())
        }
        save_json(eval_result, eval_file)
        
        print(f"✅ B1-H model completed successfully for {rung}")
        print(f"📁 Saved to {output_file}")
        return True
        
    except Exception as e:
        print(f"❌ Direct execution failed: {e}")
        print(f"🔧 Falling back to subprocess method...")
        return False

## Cell 3b: Import Baseline Scripts Directly

Instead of calling external processes, we'll import the baseline scripts directly for better integration with Jupyter.

In [ ]:
# Import the baseline scripts directly instead of using subprocess
import sys
from pathlib import Path

# Add the project root to Python path so we can import the modules
project_root = Path('.').resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

try:
    # Import the baseline functions directly
    from atus_analysis.scripts.common_hier import (
        prepare_long_with_groups, pool_rare_quarter, 
        save_json, nll_b1, fit_b1_hier, parse_time_blocks
    )
    print("Successfully imported baseline common functions")
    
    
except ImportError as e:
    print(f"Could not import baseline functions directly: {e}")
    print("Will fall back to subprocess calls")
    USE_SUBPROCESS = True
else:
    USE_SUBPROCESS = False

✓ Successfully imported baseline common functions


## Check System Status and Prerequisites

In [32]:
print("Checking system status and prerequisites...\n")

# Check system resources
resources_ok = check_system_resources()

print("\n=== File Prerequisites ===")
# Check required files
required_files = [
    SEQUENCES_FILE,
    SUBGROUPS_FILE,
    "atus_analysis/scripts/baseline1_hier.py",
    "atus_analysis/scripts/baseline2_hier.py"
]

files_ok = True
for file_path in required_files:
    if Path(file_path).exists():
        print(f"✓ {file_path}")
    else:
        print(f"✗ {file_path} - NOT FOUND")
        files_ok = False

# Check output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"✓ Output directory: {OUTPUT_DIR}")

# Load and display current progress
print("\n=== Current Progress ===")
progress = load_progress()
completed = progress.get('completed_rungs', [])
failed = progress.get('failed_rungs', [])

print(f"Completed rungs: {completed if completed else 'None'}")
print(f"Failed rungs: {failed if failed else 'None'}")
print(f"Remaining rungs: {[r for r in RUNG_SPECS.keys() if r not in completed]}")

# Overall status
print("\n=== Overall Status ===")
if resources_ok and files_ok:
    print("✅ READY TO START EXPERIMENTS")
    print("\nYou can now run the individual experiment cells below.")
else:
    print("❌ PREREQUISITES NOT MET")
    if not resources_ok:
        print("   - System resources may be insufficient")
    if not files_ok:
        print("   - Required files are missing")
    print("\nPlease resolve issues before continuing.")

Checking system status and prerequisites...

=== System Resources ===
Memory: 1.2% used, 745.8GB available
CPU: 0.0% usage
Disk: 1941552.2GB free

✓ System resources look good

=== File Prerequisites ===
✓ atus_analysis/data/sequences/markov_sequences.parquet
✓ atus_analysis/data/processed/subgroups.parquet
✓ atus_analysis/scripts/baseline1_hier.py
✓ atus_analysis/scripts/baseline2_hier.py
✓ Output directory: atus_analysis/data/models

=== Current Progress ===
Completed rungs: ['R1', 'R2', 'R3', 'R4', 'R5']
Failed rungs: ['R1', 'R6', 'R6', 'R6']
Remaining rungs: ['R6', 'R7']

=== Overall Status ===
✅ READY TO START EXPERIMENTS

You can now run the individual experiment cells below.


# Individual Experiment Cells

## Instructions for Running Experiments

**Run these cells ONE AT A TIME** in order. Each cell represents one complete experiment rung.

- **Safe to interrupt**: You can stop any cell with the stop button - progress is automatically saved
- **Resume anytime**: If you restart the kernel, just re-run cells 1-4, then continue from where you left off
- **Skip completed**: Cells will automatically skip rungs that have already completed successfully
- **Monitor progress**: Each cell shows detailed progress and resource usage

---

## Cell 5: Run Experiment R1 (Region Only)

**Expected runtime: 30-60 minutes**  
**Memory usage: Low-Medium**  
**Description: Simplest model - groups by region only**

In [ ]:
# Run R1 experiment
success = run_single_rung("R1", include_hazard=False)

if success:
    print("\nR1 completed successfully! You can now run R2.")
else:
    print("\nR1 failed. Check the error messages above and try again.")


🚀 STARTING RUNG R1
📋 Rung: R1
📋 Groupby: region
📁 Output directory: atus_analysis/data/models/R1
⚡ Include hazard: False

🔍 Checking system resources...
=== System Resources ===
Memory: 1.1% used, 746.5GB available
CPU: 0.0% usage
Disk: 1941548.8GB free

✓ System resources look good

--- 📊 Step 1: B1-H Model for R1 ---
🎯 Attempting direct execution...
📊 Loading data for R1...
✓ Loaded 36404352 sequences and 252808 subgroups
📂 Loading existing split from atus_analysis/data/models/fixed_split.parquet
🔄 Preparing data with groupby: region
❌ Direct execution failed: prepare_long_with_groups() missing 1 required positional argument: 'blocks'
🔄 Falling back to subprocess method...
🖥️  Using subprocess execution...
🖥️  Running B1-H via subprocess for R1...
Command: /sw/eb/sw/Anaconda3/2023.09-0/bin/python -m atus_analysis.scripts.baseline1_hier --sequences atus_analysis/data/sequences/markov_sequences.parquet --subgroups atus_analysis/data/processed/subgroups.parquet --out_dir atus_analysis/

## Cell 6: Run Experiment R2 (Region + Sex)

**Expected runtime: 30-60 minutes**  
**Memory usage: Low-Medium**  
**Description: Groups by region and sex**

In [ ]:
# Run R2 experiment
success = run_single_rung("R2", include_hazard=False)

if success:
    print("\nR2 completed successfully! You can now run R3.")
else:
    print("\nR2 failed. Check the error messages above and try again.")


🚀 STARTING RUNG R2
📋 Rung: R2
📋 Groupby: region,sex
📁 Output directory: atus_analysis/data/models/R2
⚡ Include hazard: False

🔍 Checking system resources...
=== System Resources ===
Memory: 1.2% used, 746.0GB available
CPU: 0.0% usage
Disk: 1941548.8GB free

✓ System resources look good

--- 📊 Step 1: B1-H Model for R2 ---
🎯 Attempting direct execution...
📊 Loading data for R2...
✓ Loaded 36404352 sequences and 252808 subgroups
📂 Loading existing split from atus_analysis/data/models/fixed_split.parquet
🔄 Preparing data with groupby: region,sex
❌ Direct execution failed: prepare_long_with_groups() missing 1 required positional argument: 'blocks'
🔄 Falling back to subprocess method...
🖥️  Using subprocess execution...
🖥️  Running B1-H via subprocess for R2...
Command: /sw/eb/sw/Anaconda3/2023.09-0/bin/python -m atus_analysis.scripts.baseline1_hier --sequences atus_analysis/data/sequences/markov_sequences.parquet --subgroups atus_analysis/data/processed/subgroups.parquet --out_dir atus_a

## Cell 7: Run Experiment R3 (Region + Employment)

**Expected runtime: 30-60 minutes**  
**Memory usage: Medium**  
**Description: Groups by region and employment status**

In [ ]:
success = run_experiment_safely(
    'R3',
    lambda: run_single_experiment_wrapper(3, dataset_path, args, 
                                         compute_expected=True)
)

if success:
    print("\nR3 completed successfully! You can now run R4.")
else:
    print("\nR3 failed. Check the error messages above and try again.")


🚀 STARTING RUNG R3
📋 Rung: R3
📋 Groupby: region,employment
📁 Output directory: atus_analysis/data/models/R3
⚡ Include hazard: False

🔍 Checking system resources...
=== System Resources ===
Memory: 1.2% used, 745.9GB available
CPU: 0.0% usage
Disk: 1941548.8GB free

✓ System resources look good

--- 📊 Step 1: B1-H Model for R3 ---
🎯 Attempting direct execution...
📊 Loading data for R3...
✓ Loaded 36404352 sequences and 252808 subgroups
📂 Loading existing split from atus_analysis/data/models/fixed_split.parquet
🔄 Preparing data with groupby: region,employment
❌ Direct execution failed: prepare_long_with_groups() missing 1 required positional argument: 'blocks'
🔄 Falling back to subprocess method...
🖥️  Using subprocess execution...
🖥️  Running B1-H via subprocess for R3...
Command: /sw/eb/sw/Anaconda3/2023.09-0/bin/python -m atus_analysis.scripts.baseline1_hier --sequences atus_analysis/data/sequences/markov_sequences.parquet --subgroups atus_analysis/data/processed/subgroups.parquet --

## Cell 8: Run Experiment R4 (Region + Day Type)

**Expected runtime: 30-60 minutes**  
**Memory usage: Medium**  
**Description: Groups by region and day type (weekday/weekend)**

In [ ]:
success = run_experiment_safely(
    'R4',
    lambda: run_single_experiment_wrapper(4, dataset_path, args, 
                                         compute_expected=True)
)

if success:
    print("\nR4 completed successfully! You can now run R5.")
else:
    print("\nR4 failed. Check the error messages above and try again.")


🚀 STARTING RUNG R4
📋 Rung: R4
📋 Groupby: region,day_type
📁 Output directory: atus_analysis/data/models/R4
⚡ Include hazard: False

🔍 Checking system resources...
=== System Resources ===
Memory: 1.2% used, 745.9GB available
CPU: 0.0% usage
Disk: 1941548.7GB free

✓ System resources look good

--- 📊 Step 1: B1-H Model for R4 ---
🎯 Attempting direct execution...
📊 Loading data for R4...
✓ Loaded 36404352 sequences and 252808 subgroups
📂 Loading existing split from atus_analysis/data/models/fixed_split.parquet
🔄 Preparing data with groupby: region,day_type
❌ Direct execution failed: prepare_long_with_groups() missing 1 required positional argument: 'blocks'
🔄 Falling back to subprocess method...
🖥️  Using subprocess execution...
🖥️  Running B1-H via subprocess for R4...
Command: /sw/eb/sw/Anaconda3/2023.09-0/bin/python -m atus_analysis.scripts.baseline1_hier --sequences atus_analysis/data/sequences/markov_sequences.parquet --subgroups atus_analysis/data/processed/subgroups.parquet --out_

## Cell 9: Run Experiment R5 (Region + Household Size)

**Expected runtime: 60-90 minutes**  
**Memory usage: Medium**  
**Description: Groups by region and household size band**

In [ ]:
success = run_experiment_safely(
    'R5',
    lambda: run_single_experiment_wrapper(5, dataset_path, args, 
                                         compute_expected=True)
)

if success:
    print("\nR5 completed successfully! You can now run R6.")
else:
    print("\nR5 failed. Check the error messages above and try again.")


🚀 STARTING RUNG R5
📋 Rung: R5
📋 Groupby: region,hh_size_band
📁 Output directory: atus_analysis/data/models/R5
⚡ Include hazard: False

🔍 Checking system resources...
=== System Resources ===
Memory: 1.2% used, 745.9GB available
CPU: 0.0% usage
Disk: 1941548.6GB free

✓ System resources look good

--- 📊 Step 1: B1-H Model for R5 ---
🎯 Attempting direct execution...
📊 Loading data for R5...
✓ Loaded 36404352 sequences and 252808 subgroups
📂 Loading existing split from atus_analysis/data/models/fixed_split.parquet
🔄 Preparing data with groupby: region,hh_size_band
❌ Direct execution failed: prepare_long_with_groups() missing 1 required positional argument: 'blocks'
🔄 Falling back to subprocess method...
🖥️  Using subprocess execution...
🖥️  Running B1-H via subprocess for R5...
Command: /sw/eb/sw/Anaconda3/2023.09-0/bin/python -m atus_analysis.scripts.baseline1_hier --sequences atus_analysis/data/sequences/markov_sequences.parquet --subgroups atus_analysis/data/processed/subgroups.parque

## Cell 9b: Add Quarter Column to Subgroups File (Run Before R6)

**Important**: Run this cell before attempting R6 or R7 experiments.  
This will permanently add the quarter column to the subgroups.parquet file.  
**Runtime**: 1-2 minutes  
**Purpose**: Ensures R6 and R7 experiments have the required quarter column

In [ ]:
success = run_experiment_safely(
    'R6',
    lambda: run_single_experiment_wrapper(6, dataset_path, args, 
                                         compute_expected=True)
)

if success:
    print("\nR6 completed successfully! You can now run R7.")
else:
    print("\nR6 failed. Check the error messages above and try again.")

🗓️  Adding quarter column to subgroups.parquet file...
📂 Loading subgroups from: atus_analysis/data/processed/subgroups.parquet
✅ Loaded 252808 subgroup records
Current columns: ['TUCASEID', 'sex', 'hh_size_band', 'month', 'region', 'employment', 'day_type', 'TUFNWGTP']
📝 Quarter column not found - adding it now...
🗓️  Adding quarter column from month data...
✓ Quarter column added. Distribution:
   Q1: 68,017 respondents
   Q2: 62,107 respondents
   Q3: 61,880 respondents
   Q4: 60,804 respondents
💾 Creating backup at: atus_analysis/data/processed/subgroups.parquet.backup
💾 Saving updated subgroups with quarter column...
✅ Quarter column successfully added to subgroups.parquet!
Final quarter distribution:
   Q1: 68,017 respondents
   Q2: 62,107 respondents
   Q3: 61,880 respondents
   Q4: 60,804 respondents

📋 Summary:
   - Original file backed up to: atus_analysis/data/processed/subgroups.parquet.backup
   - Updated file saved to: atus_analysis/data/processed/subgroups.parquet
   - Q

## Cell 10: Run Experiment R6 (Full Routing Model)

**Expected runtime: 90-120 minutes**  
**Memory usage: High**  
**Description: Full complexity routing model with all demographic variables**

In [ ]:
success = run_experiment_safely(
    'R7',
    lambda: run_single_experiment_wrapper(7, dataset_path, args, 
                                         compute_expected=True)
)

if success:
    print("\nR7 completed successfully! All experiments finished.")
else:
    print("\nR7 failed. Check the error messages above and try again.")


🚀 STARTING RUNG R6
📋 Rung: R6
📋 Groupby: employment,day_type,hh_size_band,sex,region,quarter
📁 Output directory: atus_analysis/data/models/R6
⚡ Include hazard: False

🔍 Checking system resources...
=== System Resources ===
Memory: 1.2% used, 745.8GB available
CPU: 0.0% usage
Disk: 1941552.2GB free

✓ System resources look good

--- 📊 Step 1: B1-H Model for R6 ---
🎯 Attempting direct execution...
📊 Loading data for R6...
✓ Quarter column already exists
✓ Loaded 36404352 sequences and 252808 subgroups
✓ Parsed time blocks: [('night', 0, 5), ('morning', 6, 11), ('afternoon', 12, 17), ('evening', 18, 23)]
📂 Loading existing split from atus_analysis/data/models/fixed_split.parquet
🔄 Preparing data with groupby: employment,day_type,hh_size_band,sex,region,quarter
❌ Direct execution failed: prepare_long_with_groups() missing 1 required positional argument: 'blocks'
🔧 Falling back to subprocess method...
🖥️  Using subprocess execution...
🖥️  Running B1-H via subprocess for R6...
Command: /sw/

## Cell 11: Run Experiment R7 (Full Model + Hazard)

**Expected runtime: 120-180 minutes**  
**Memory usage: High**  
**Description: Full model including hazard modeling - most computationally intensive**

In [ ]:
print("Helper functions defined with improved direct execution support")


🚀 STARTING RUNG R7
📋 Rung: R7
📋 Groupby: employment,day_type,hh_size_band,sex,region,quarter
📁 Output directory: atus_analysis/data/models/R7
⚡ Include hazard: True

🔍 Checking system resources...
=== System Resources ===
Memory: 1.2% used, 745.8GB available
CPU: 0.0% usage
Disk: 1941553.0GB free

✓ System resources look good

--- 📊 Step 1: B1-H Model for R7 ---
🎯 Attempting direct execution...
📊 Loading data for R7...
✓ Quarter column already exists
✓ Loaded 36404352 sequences and 252808 subgroups
✓ Parsed time blocks: [('night', 0, 5), ('morning', 6, 11), ('afternoon', 12, 17), ('evening', 18, 23)]
📂 Loading existing split from atus_analysis/data/models/fixed_split.parquet
🔄 Preparing data with groupby: employment,day_type,hh_size_band,sex,region,quarter
❌ Direct execution failed: prepare_long_with_groups() missing 1 required positional argument: 'blocks'
🔧 Falling back to subprocess method...
🖥️  Using subprocess execution...
🖥️  Running B1-H via subprocess for R7...
Command: /sw/e

## Cell 12: Final Summary and Results

In [ ]:
# Final validation and summary
import json
from pathlib import Path

# Check completion status
all_rungs = ["R1", "R2", "R3", "R4", "R5", "R6", "R7"] 
completed = []
failed = []

for rung in all_rungs:
    expected_file = OUTPUT_DIR / rung / "run_summary.json"
    if expected_file.exists():
        try:
            with open(expected_file) as f:
                summary = json.load(f)
                if summary.get('status') == 'completed':
                    completed.append(rung)
                else:
                    failed.append(rung)
        except:
            failed.append(rung)

print(f"=== Final Experiment Status ===")
print(f"Completed: {completed}")
print(f"Failed/Incomplete: {failed}")

# Show timing information
print(f"\n=== Runtime Summary ===")
total_time = 0
for rung in completed:
    duration_key = f'{rung}_duration_seconds'
    if duration_key in progress:
        duration = progress[duration_key]
        total_time += duration
        print(f"{rung}: {duration:.0f} seconds ({duration/60:.1f} minutes)")

if total_time > 0:
    print(f"\nTotal runtime: {total_time:.0f} seconds ({total_time/60:.1f} minutes, {total_time/3600:.1f} hours)")

# Show output locations
print("\n=== Output Files ===")
for rung in completed:
    rung_dir = OUTPUT_DIR / rung
    if rung_dir.exists():
        files = list(rung_dir.glob("*.json"))
        print(f"{rung}: {len(files)} files in {rung_dir}")

# Success rate
if len(completed) + len(failed) > 0:
    success_rate = len(completed) / 7 * 100
    print(f"\nOverall success rate: {success_rate:.1f}%")

if len(completed) == len(all_rungs):
    print("\nALL EXPERIMENTS COMPLETED SUCCESSFULLY!")
elif len(failed) > 0:
    print(f"\nSome experiments failed. You can re-run the failed cells to retry.")
else:
    print(f"\n{len(all_rungs) - len(completed)} experiments remaining.")

print(f"\nProgress file saved as: {PROGRESS_FILE}")
print(f"Output directory: {OUTPUT_DIR}")


FINAL EXPERIMENT SUMMARY

Total rungs: 7
Completed: 7 - ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7']
Failed: 6 - ['R1', 'R6', 'R6', 'R6', 'R6', 'R7']
Not attempted: []

=== Timing Information ===
R1: 1094 seconds (18.2 minutes)
R2: 1537 seconds (25.6 minutes)
R3: 1525 seconds (25.4 minutes)
R4: 1528 seconds (25.5 minutes)
R5: 1525 seconds (25.4 minutes)
R6: 2185 seconds (36.4 minutes)
R7: 4735 seconds (78.9 minutes)

Total runtime: 14129 seconds (235.5 minutes, 3.9 hours)

=== Output Files ===
R1: 2 files in atus_analysis/data/models/R1
R2: 2 files in atus_analysis/data/models/R2
R3: 2 files in atus_analysis/data/models/R3
R4: 2 files in atus_analysis/data/models/R4
R5: 2 files in atus_analysis/data/models/R5
R6: 2 files in atus_analysis/data/models/R6
R7: 4 files in atus_analysis/data/models/R7

Overall success rate: 100.0%

🎉 ALL EXPERIMENTS COMPLETED SUCCESSFULLY! 🎉

Progress file saved as: experiment_progress_jupyter.json
Output directory: atus_analysis/data/models
